In [4]:
from pathlib import Path

%reload_ext autoreload
%autoreload 2

from scenario_functions import load_energyplan_file, format_value, build_params

# Create Scenario file

This notebook takes the `IDA2045_nuclear_flexible.txt` and changes only the inputs, where our analysis differs. The idea behind this is clarity and overview:

## 1. Cases

### 1.1. Specify changes

First, I establish the base parameters and the cases, which is comprised of the following changes.

- Nb. FOM is to be enteres as a share of investment costs. As our assumptions, regarding O&M, are the same as the norwegians (investment cost: 6.18, O&M: 1.82%), then we compute ours as: $(6.18 \cdot 0.0182) / 4.36 = 0.026$.
- Using a similar argument, we change O&M for offshore wind, where we assume FOA's O&M, but not their investment cost: $(1.9\cdot 0.0251) / 2.5=0.019$.

In [5]:
# 1. basic changes
base_params = {

    # variables --------------- # value ------- # description --------------------- # source -------------------
 
    # demand
    'Input_el_demand_Twh'       : 311/10,       # electricity demand                # KF25
    'Input_add_el_TWh'          : 156/10,       # additional electricity demand     # KF25

    # production
    'input_RES1_capacity'       : 5150,         # on-shore wind                     # KF25
    'input_RES3_capacity'       : 25134,        # solar PV                          # KF25
    
    # variation patterns
    'Filnavn_elbehov'           : '2024_ENS_ProductionConsumptionSettlement.txt',   # ENS_2024
    'Filnavn_wind'              : 'KF25_2035_Havvind.txt',                          # KF25
    'Filnavn_wave'              : 'KF25_2035_Landvind.txt',                         # KF25
    'Filnavn_pv'                : 'KF25_2035_Solceller.txt',                        # KF25
    'Filnavn_prices'            : 'KF25_2035_Timepriser.txt',                       # KF25
    # 'filnavn_nuclear'           : 'el_atomkraft_dh.txt',  
    # 'Filnavn_cshp'              : 'varme_atomkraft_dh.txt',  # alternative is nuclear=const.txt 

    # optimization
    # 'input_max_imp_exp'         : 100000,       # transmission line capacity      # KF25

    # costs
    'input_Inv_Nuclear'         : 436/100,      # investment cost nuclear           # OECD IEA
    'input_Period_Nuclear'      : 60,           # economic lifetime nuclear         # FOA
    'input_FOM_Nuclear'         : 26/10,        # fixed O&M cost nuclear            # OECD IEA
    'input_Inv_WindOffshore'    : 25/10,        # investment cost offshore wind     # FL
    'input_Period_WindOffshore' : 30,           # economic lifetime offshore wind   # FOA
    'input_FOM_WindOffshore'    : 19/10,        # fixed O&M cost offshore wind      # FOA
}

# 2. base scenario
base_case_params = {
    'input_nuclear_cap'         : 0,            # base nuclear capacity             # FL + LL
    'input_RES2_capacity'       : 8287,         # base off-shore wind               # KF25 + FL
    'input_keol_reg'            : 234500000,                                        # NOR_1GW (FOA: 923457000)
}
shock_case_params = {
    'input_nuclear_cap'         : 1000,         # shock nuclear capacity            # FL + LL
    'input_RES2_capacity'       : 6287,         # shock off-shore wind              # KF25 + FL
    'input_keol_reg'            : 234597000,                                        # FOA IDA
}

### 1.2 Build outputfile

In [6]:
ref_path = r'..\ZipEnergyPLAN163\energyPlan Data\Data\IDA2045_nuclear_flexible.txt'
out_path = r'..\ZipEnergyPLAN163\energyPlan Data\Data\R_1_nt.txt'

params = build_params(
    case="base",
    base_params=base_params,
    base_case_params=base_case_params,
    shock_case_params=shock_case_params,
)

lines, value_idx = load_energyplan_file(ref_path)

for name, new_val in params.items():
    if name not in value_idx:
        raise KeyError(f"Parameter name not found in file: {name!r}")
    idx = value_idx[name]
    lines[idx] = format_value(new_val)

with open(out_path, "w", encoding="utf-16") as f:
    f.writelines(lines)